Stuff Documents 체인을 사용하여 완전한 RAG 파이프라인을 구현하세요.

체인을 수동으로 구현해야 합니다.

체인에 ConversationBufferMemory를 부여합니다.

이 문서를 사용하여 RAG를 수행하세요: https://gist.github.com/serranoarevalo/5acf755c2b8d83f1707ef266b82ea223

체인에 다음 질문을 합니다:

Aaronson 은 유죄인가요?

그가 테이블에 어떤 메시지를 썼나요?

Julia 는 누구인가요?

In [18]:
import sys
import os
sys.path.append('../../')
from get_env import get_env
import requests
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationBufferMemory
from langchain.schema import BaseMessage, HumanMessage, AIMessage
from typing import List, Dict, Any

# API 키 설정
api_key = get_env()


In [19]:
# 문서 로딩 및 분할
loader = TextLoader("../../files/chapter_three.txt", encoding="utf-8")
documents = loader.load()

# 텍스트 분할기 설정
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

# 문서 분할
docs = splitter.split_documents(documents)
print(f"문서가 {len(docs)}개의 청크로 분할되었습니다.")


문서가 34개의 청크로 분할되었습니다.


In [20]:
# 임베딩 및 벡터스토어 설정
cache_dir = LocalFileStore("./.cache/")
embeddings = OpenAIEmbeddings()
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

# FAISS 벡터스토어 생성
vectorstore = FAISS.from_documents(docs, cached_embeddings)
retriever = vectorstore.as_retriever()

print("벡터스토어 생성 완료")


벡터스토어 생성 완료


In [21]:
# LLM 및 메모리 설정
llm = ChatOpenAI(temperature=0.1)
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

# 프롬프트 템플릿 설정
prompt = ChatPromptTemplate.from_messages([
    ("system", """당신은 도움이 되는 어시스턴트입니다. 주어진 컨텍스트를 사용하여 질문에 답하세요. 
    답을 모르면 모른다고 말하고, 답을 만들어내지 마세요.
    
    컨텍스트: {context}
    
    대화 기록: {chat_history}"""),
    ("human", "{question}")
])

print("LLM 및 메모리 설정 완료")


LLM 및 메모리 설정 완료


In [22]:
# Stuff Documents 체인 수동 구현
class StuffDocumentsChain:
    def __init__(self, retriever, prompt, llm, memory):
        self.retriever = retriever
        self.prompt = prompt
        self.llm = llm
        self.memory = memory
    
    def _get_relevant_documents(self, question: str) -> str:
        """관련 문서들을 검색하고 결합"""
        docs = self.retriever.get_relevant_documents(question)
        return "\n\n".join([doc.page_content for doc in docs])
    
    def invoke(self, question: str) -> str:
        """질문에 대한 답변 생성"""
        # 관련 문서 검색
        context = self._get_relevant_documents(question)
        
        # 메모리에서 대화 기록 가져오기
        chat_history = self.memory.chat_memory.messages
        
        # 프롬프트에 입력값 설정
        formatted_prompt = self.prompt.format(
            context=context,
            question=question,
            chat_history=chat_history
        )
        
        # LLM으로 답변 생성
        response = self.llm.invoke(formatted_prompt)
        
        # 메모리에 대화 저장
        self.memory.chat_memory.add_user_message(question)
        self.memory.chat_memory.add_ai_message(response.content)
        
        return response.content

# 체인 생성
chain = StuffDocumentsChain(retriever, prompt, llm, memory)
print("Stuff Documents 체인 생성 완료")


Stuff Documents 체인 생성 완료


In [23]:
# 질문 1: Aaronson은 유죄인가요?
print("=== 질문 1: Aaronson은 유죄인가요? ===")
answer1 = chain.invoke("Aaronson은 유죄인가요?")
print(answer1)
print("\n")


=== 질문 1: Aaronson은 유죄인가요? ===
System: 네, Jones, Aaronson 및 Rutherford는 자신들에게 적용된 범죄로 유죄 판결을 받았습니다.




In [24]:
# 질문 2: 그가 테이블에 어떤 메시지를 썼나요?
print("=== 질문 2: 그가 테이블에 어떤 메시지를 썼나요? ===")
answer2 = chain.invoke("그가 테이블에 어떤 메시지를 썼나요?")
print(answer2)
print("\n")


=== 질문 2: 그가 테이블에 어떤 메시지를 썼나요? ===
System: 책에서 언급된 내용에는 Aaronson이 테이블에 어떤 메시지를 썼는지에 대한 구체적인 정보가 포함되어 있지 않습니다.




In [25]:
# 질문 3: Julia는 누구인가요?
print("=== 질문 3: Julia는 누구인가요? ===")
answer3 = chain.invoke("Julia는 누구인가요?")
print(answer3)
print("\n")


=== 질문 3: Julia는 누구인가요? ===
System: Julia는 책에서 언급된 다른 주요 캐릭터 중 하나로, 윈스턴과 함께 이야기의 중심에 있는 여성입니다. 그녀는 윈스턴의 사랑이자 동료로 등장합니다.




In [26]:
# 메모리 상태 확인
print("=== 대화 기록 ===")
print(memory.chat_memory.messages)


=== 대화 기록 ===
[HumanMessage(content='Aaronson은 유죄인가요?'), AIMessage(content='System: 네, Jones, Aaronson 및 Rutherford는 자신들에게 적용된 범죄로 유죄 판결을 받았습니다.'), HumanMessage(content='그가 테이블에 어떤 메시지를 썼나요?'), AIMessage(content='System: 책에서 언급된 내용에는 Aaronson이 테이블에 어떤 메시지를 썼는지에 대한 구체적인 정보가 포함되어 있지 않습니다.'), HumanMessage(content='Julia는 누구인가요?'), AIMessage(content='System: Julia는 책에서 언급된 다른 주요 캐릭터 중 하나로, 윈스턴과 함께 이야기의 중심에 있는 여성입니다. 그녀는 윈스턴의 사랑이자 동료로 등장합니다.')]
